In [1]:
import pandas as pd
import numpy as np
import glob


In [2]:
def avg_last_execution(files_path, drift_detector, generator, metric = 'detected changes'):
    
    results = []
    for file in files_path:
        #print(file)
        df = pd.read_csv(file)
        #print(df.iloc[-1][metric])
        value = df.iloc[-1][metric]
        if value != "?":
            results.append(round(float(value), 2))
        
    
    tuple_results = tuple([np.mean(results), np.std(results), drift_detector, generator])
   
    #print(f'avg of all last execution: {np.mean(median)}')
    return tuple_results 

In [3]:
drift_detectors_params = [
                            {"id": "DDM", "drift_name": "DDM", "params": ""},    \
                            {"id": "EDDM", "drift_name": "EDDM", "params": ""},   \
                            {"id": "ADWIN", "drift_name": "ADWINChangeDetector", "params": ""},  \
                            {"id": "ECDD", "drift_name": "EWMAChartDM", "params": ""},   \
                            {"id": "STEPD", "drift_name": "STEPD", "params": ""},  \
                            {"id": "SeqDrift2", "drift_name": "SeqDrift2ChangeDetector", "params": ""},  \
                            {"id": "SEED", "drift_name": "SEEDChangeDetector", "params": ""},   \
                            {"id": "HDDM_A_Test", "drift_name": "HDDM_A_Test", "params": ""},  \
                            {"id": "HDDM_W_Test", "drift_name": "HDDM_W_Test", "params": ""},  \
                            #{"id": "FHDDM", "drift_name": "FHDDM", "params": ""},  \
                            #{"id": "FTDD", "drift_name": "FTDD", "params": ""},   \
                            {"id": "RDDM_30", "drift_name": "RDDM", "params": "-n 30 -w 2 -o 3"}, \
                            {"id": "RDDM","drift_name": "RDDM", "params": ""},   \
                            #{"id": "WSTD","drift_name": "WSTD", "params": ""},    \
                            {"id": "HypothesisTestDetectorKs","drift_name": "HypothesisTestDetector", "params": "-h ks"},    \
                            {"id": "HypothesisTestDetectorWrs","drift_name": "HypothesisTestDetector", "params": "-h wrs"},    \
                            {"id": "HypothesisTestDetectorTt","drift_name": "HypothesisTestDetector", "params": "-h tt"},    \
                            {"id": "MultivariateDetector","drift_name": "moa.learners.ChangeDetectorEnsembleMultivariateLearner", "params": "-d (PageHinkleyDM -d 0.002) -p 20"},    \
                            {"id": "Studd","drift_name": "moa.learners.StuddLearner", "params": "-b (meta.AdaptiveRandomForest -u  -q -w) -s (meta.AdaptiveRandomForest -u  -q -w) -d (PageHinkleyDM -d 0.002) -w 500"},    \
                            {"id": "NoChangeDetector","drift_name": "NoChangeDetectorNaive", "params": "-n 1000"}                        
                        ]

# Table Abrupt HT 100K

In [7]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 95
type_drift = "Abrupt"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "detected changes"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,64.133333,54.533333,183.866667,300.700000,49.800000,80.333333,356.700000,310.700000,1699.833333,61.433333,89.766667,85.066667,64.733333,81.433333,81.066667,42.733333,0.0
AGRAW2,94.533333,155.700000,243.833333,213.766667,85.766667,89.333333,193.466667,195.333333,3048.000000,61.433333,89.766667,86.200000,75.966667,163.866667,211.300000,149.000000,0.1
LED,0.000000,0.000000,68.233333,3.933333,3.466667,24.133333,338.033333,305.033333,2432.766667,0.000000,89.766667,0.000000,20.233333,396.133333,147.533333,246.366667,5.4
MIXED,91.233333,82.733333,169.533333,149.433333,71.933333,89.366667,165.433333,160.533333,2746.400000,10.266667,89.766667,86.100000,68.833333,216.633333,87.433333,170.200000,0.0
RANDOM_RBF,152.800000,75.700000,104.766667,109.866667,78.866667,82.500000,219.000000,264.366667,2662.733333,0.000000,89.766667,91.466667,70.433333,581.300000,302.733333,286.300000,19.8
SINE,80.866667,109.033333,209.133333,223.000000,84.233333,82.666667,185.000000,210.300000,2825.200000,0.333333,89.766667,98.233333,78.633333,177.766667,125.700000,172.133333,0.0
WAVEFORM,0.000000,2.000000,147.166667,68.133333,1.100000,3.100000,296.200000,276.000000,2693.633333,0.000000,89.766667,4.166667,0.700000,3.866667,26.033333,0.700000,0.0


# Table Gradual HT 100K

In [12]:
classifier = "trees.HoeffdingTree"
drift_size = 100000
#partialy_drift options: 95, 96, 97, 98, 99
partialy_drift = 95
type_drift = "Gradual"
#metric name options: detected changes, detected warnings, delay detection (average), delay true detection (average), MTFA (average), MDR, true changes detected, 
metric_name = "detected changes"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        ##trees.HoeffdingTree_ADWIN_100000_95_0_AGRAW1_Abrupt
        files_path = glob.glob(f'../results/{drift_key}/{classifier}_{drift["id"]}_{drift_size}_{partialy_drift}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value, metric = metric_name)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,HypothesisTestDetectorKs,HypothesisTestDetectorTt,HypothesisTestDetectorWrs,MultivariateDetector,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2,Studd
generator,,,,,,,,,,,,,,,,,
AGRAW1,65.033333,54.600000,183.833333,312.933333,53.666667,81.800000,356.566667,291.733333,1753.233333,56.6,89.766667,75.333333,43.433333,71.400000,77.500000,44.966667,0.000000
AGRAW2,99.266667,182.500000,241.566667,209.466667,74.200000,96.333333,198.366667,199.833333,3051.600000,56.6,89.766667,71.266667,73.366667,158.866667,208.266667,157.666667,0.000000
LED,0.000000,0.000000,66.866667,3.933333,0.666667,18.866667,356.900000,308.333333,2372.666667,0.0,89.766667,0.000000,0.000000,418.833333,187.233333,227.500000,0.133333
MIXED,99.100000,91.066667,174.700000,131.566667,86.433333,86.400000,115.133333,136.933333,2662.000000,7.6,89.766667,98.233333,89.133333,221.133333,103.900000,175.133333,0.000000
RANDOM_RBF,156.166667,82.000000,102.933333,112.966667,73.666667,76.166667,242.700000,238.366667,2660.066667,0.0,89.766667,82.033333,77.433333,573.433333,308.066667,265.500000,30.633333
SINE,81.800000,112.333333,211.266667,234.433333,79.066667,79.466667,162.166667,180.566667,2875.366667,0.0,89.766667,105.866667,94.533333,182.366667,118.933333,167.266667,0.000000
WAVEFORM,0.000000,2.233333,166.966667,100.566667,1.266667,3.633333,260.500000,291.133333,2694.966667,0.0,89.766667,7.600000,3.066667,4.433333,27.000000,0.700000,0.200000
